In [1]:
# !pip install tabpfn --no-index --find-links=file:///kaggle/input/tab-pfn-dataset
# !mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
# !cp /kaggle/input/tab-pfn-dataset/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [2]:
import sys
sys.path.append('/kaggle/input/iter-strat/iter_strat')

import math

import numpy as np
import pandas as pd

import lightgbm as lgb
import catboost as cat
from catboost import Pool
import xgboost as xgb
from tabpfn import TabPFNClassifier

import itertools
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits, BaseShuffleSplit, _validate_shuffle_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import log_loss
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.utils import check_random_state
from sklearn.utils.validation import _num_samples, check_array
from sklearn.utils.multiclass import type_of_target

from scipy import stats

import eli5
from IPython.display import display
from eli5.permutation_importance import get_score_importances
from eli5.sklearn import PermutationImportance

import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import seaborn as sns

import optuna

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


from colorama import Style, Fore

palette = ['#302c36', '#037d97', '#E4591E', '#C09741',
           '#EC5B6D', '#90A6B1', '#6ca957', '#D8E3E2']

blk = Style.BRIGHT + Fore.BLACK
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL


class CFG:
    # main
    kaggle = False
    test = False
    
    # features
    fe_drop = True
    
    del_errs = False
    del_outliers = True
    del_outliers_adj = False
    
    feature_sel = False
    n_feature_sel_repeats = 5
    n_feature_sel_folds = 5
    
    undersample = False
    oversample = False
    
    nan_impute = False
    standard_scale = False
    log = False
    
    # optimization
    n_estimators = 3000
    early_stopping_rounds = 100
    
    lgbm_optimize = True
    xgb_optimize = False
    cb_optimize = False
    
    n_trials = 500
    n_optimize_folds = 5
    n_optimize_repeats = 2
    
    # train
    lgbm_train = True
    xgb_train = False
    cb_train = False
    tabpfn_train = False

    # inference
    n_stacking_folds = 5
    n_stacking_models_lgbm = 40
    n_stacking_models_xgb = 10
    n_stacking_models_cb = 20
    n_stacking_models_tabpfn = 20

    adjust_class_threshold = False
    

# Load Data

In [3]:
if CFG.kaggle:
    COMP_PATH = "/kaggle/input/icr-identify-age-related-conditions"
else:
    COMP_PATH = "icr-identify-age-related-conditions"

train_df = pd.read_csv(f'{COMP_PATH}//train.csv')
test_df = pd.read_csv(f'{COMP_PATH}/test.csv')
greeks = pd.read_csv(f"{COMP_PATH}/greeks.csv")
sample_submission = pd.read_csv(f"{COMP_PATH}/sample_submission.csv")

train_df['EJ'] = train_df['EJ'].replace({'A': 0, 'B': 1})
test_df['EJ'] = test_df['EJ'].replace({'A': 0, 'B': 1})

train_df.columns = train_df.columns.str.replace(' ', '')
test_df.columns = test_df.columns.str.replace(' ', '')

# Delete erroneus objects

In [4]:
if CFG.del_errs:
    err_objs = [292, 102, 509, 367, 313, 462, 556]
    train_df = train_df.loc[[i for i in train_df.index if i not in err_objs], :].reset_index(drop=True)
    greeks = greeks.loc[[i for i in greeks.index if i not in err_objs], :].reset_index(drop=True)

class_imbalance = 3 # train_df[train_df['Class'] == 0].shape[0] / train_df[train_df['Class'] == 1].shape[0]

# Add distance features

In [5]:
# from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier

# features = train_df.drop(['Class', 'Id'], axis=1).columns

# # average label of 20 Nearest Neighbours (colsine distance)
# knn = NearestNeighbors(n_neighbors=10, metric='cosine', n_jobs=-1)
# knn.fit(train_df[features].fillna(0))

# # train
# dists, nears = knn.kneighbors(train_df[features].fillna(0), return_distance=True)
# dists, nears = dists[:,1:], nears[:,1:]

# classes = np.array([train_df.loc[n, 'Class'] for n in nears])
# train_df['class_cos'] = np.array(classes[i].mean() for i in range(len(nears)))
# train_df['class_cos'] = train_df['class_cos'].astype(float)

# # test
# dists, nears = knn.kneighbors(test_df[features].fillna(0), return_distance=True)
# dists, nears = dists[:,1:], nears[:,1:]

# classes = np.array([train_df.loc[n, 'Class'] for n in nears])
# test_df['class_cos'] = np.array(classes[i].mean() for i in range(len(nears)))
# test_df['class_cos'] = test_df['class_cos'].astype(float)

# Drop not necessary features

In [6]:
if CFG.fe_drop:
    features = [fe for fe in train_df.columns if fe not in ['CF', 'CB', 'DV', 'BR', 'DF', 'GB', 'AH',
                                                            'CW', 'CL', 'BP', 'BD', 'FC', 'GE', 'GF',
                                                            'AR', 'GI', 'Id', 'Class', 'AX', 'DA']]
else:
    features = [fe for fe in train_df.columns if fe not in ['Id', 'Class', 'EJ']]

num_cols = train_df.select_dtypes(include=['float64']).columns

len(train_df.columns), len(features), len(num_cols)

(58, 38, 55)

# NaN impute

In [7]:
from datetime import datetime

imp = SimpleImputer(missing_values=np.nan, strategy='median')

train_df_tabpfn = train_df.copy()
test_df_tabpfn = test_df.copy()

train_df_tabpfn[num_cols] = imp.fit_transform(train_df_tabpfn[num_cols])
test_df_tabpfn[num_cols] = imp.transform(test_df_tabpfn[num_cols])

if CFG.nan_impute:
    train_df[num_cols] = train_df_tabpfn[num_cols]
    test_df[num_cols] = test_df_tabpfn[num_cols]

# Delete outliers

In [8]:
features_with_outliers = [fe for fe in train_df.columns if fe not in ['BN', 'BQ', 'CW', 'EL', 'GH', 
                                                                      'GI', 'GL', 'Id', 'Class', 'EJ']]

if CFG.del_outliers:
    for f in features_with_outliers:
        train_df[f] = train_df[f].clip(upper=train_df[f].quantile(0.99))
        test_df[f] = test_df[f].clip(upper=test_df[f].quantile(0.99))

if CFG.del_outliers_adj:
    from adjdatatools.preprocessing import AdjustedScaler

    adj_scaler = AdjustedScaler(with_centering=True)
    adj_scaler.fit(train_df[features])
    train_df[features] = adj_scaler.transform(train_df[features])
    test_df[features] = adj_scaler.transform(test_df[features])

# Standard Scale

In [33]:
sc = StandardScaler()
train_df_tabpfn[num_cols] = sc.fit_transform(train_df_tabpfn[num_cols])
test_df_tabpfn[num_cols] = sc.transform(test_df_tabpfn[num_cols])
train_df_tabpfn['EJ'] = train_df['EJ']
test_df_tabpfn['EJ'] = test_df['EJ']

if CFG.standard_scale:
    train_df[num_cols] = train_df_tabpfn[num_cols]
    test_df[num_cols] = test_df_tabpfn[num_cols]

# Log features

In [10]:
if CFG.log:
    train_df[num_cols] = np.log1p(train_df[num_cols])
    test_df[num_cols] = np.log1p(test_df[num_cols])

# for f in features:
#     train_df[f] = np.sign(train_df[f]) * np.log1p(np.abs(train_df[f])) # no significant result for LGBM

# LGBM feature selection

In [11]:
def balanced_log_loss(y_true, y_pred):

    # Nc is the number of observations
    N_1 = np.sum(y_true == 1, axis=0)
    N_0 = np.sum(y_true == 0, axis=0)

    N_inv_0 = 1/N_0 if N_0 > 0 else 0
    N_inv_1 = 1/N_1 if N_1 > 0 else 0

    # In order to avoid the extremes of the log function, each predicted probability 𝑝 is replaced with max(min(𝑝,1−10−15),10−15)
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)

    # balanced logarithmic loss
    loss_numerator = - N_inv_0 * np.sum((1 - y_true) * np.log(1 - y_pred)) - N_inv_1 * np.sum(y_true * np.log(y_pred))

    return loss_numerator / 2

if not CFG.kaggle:

    from shaphypetune import BoostBoruta

    params = {
                'n_estimators': CFG.n_estimators,
                'early_stopping_round': CFG.early_stopping_rounds,
                'objective': 'binary',
                'metric': 'logloss', 
                'n_jobs': -1,
                'is_unbalance':True, 
                'class_weight':'balanced', 
                'verbose': -1,
                'seed': 19062023,
            }

    def bll_metric(y_true, y_pred):
        return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

    def lgbm_tuning(features, permut=False, boruta=False):
        metric = balanced_log_loss
        eval_results_ = {}

        cv_scores = [] # store all cv scores of outer loop inference

        perm_df_ = pd.DataFrame()
        feature_importances_ = pd.DataFrame()
        boruta_df_ = pd.DataFrame()
        
        for i in range(CFG.n_feature_sel_repeats):
            print(f'Repeat {blu}#{i+1}')
            
            # Make random under-sampling to balance classes
            positive_count_train = train_df['Class'].value_counts()[1]
            sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train * 3, 
                                                            1: positive_count_train}, 
                                        random_state=15062023+i, 
                                        replacement=True)

            X_re, y_re = pd.concat([train_df[features], greeks.iloc[:,1:4]], axis=1), train_df['Class']
            
            if CFG.undersample:
                X_re, y_re = sampler.fit_resample(X_re, y_re)
            
            # Create Stratified Multilabel k-Fold scheme
            kf = MultilabelStratifiedKFold(n_splits=CFG.n_feature_sel_folds, shuffle=True, random_state=8062023+i)

            # Create an oof array for inner loop
            oof = np.zeros(X_re.shape[0])
            
            # Stratify based on Class and Alpha (3 types of conditions)
            for fold, (train_idx, val_idx) in enumerate(kf.split(X=X_re[features], y=X_re.iloc[:,-3:]), start = 1): 
                X, y = X_re[features], y_re

                # Split the dataset according to the fold indexes.
                X_train = X.iloc[train_idx]
                X_val = X.iloc[val_idx]
                y_train = y.iloc[train_idx]
                y_val = y.iloc[val_idx]


                X_train = X_train.reset_index(drop=True)
                y_train = y_train.reset_index(drop=True)

                # Store models here
                models_ = [] 

                eval_results_[fold]= {}

                clf = lgb.LGBMClassifier(**params)
                clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric=bll_metric, verbose=0)

                models_.append(clf)

                val_preds = clf.predict_proba(X_val)[:,1]
                oof[val_idx] = val_preds

                val_score = metric(y_val, val_preds)
                best_iter = clf.best_iteration_

                print(f'Fold: {blu}{fold:>3}{res}| {metric.__name__}: {blu}{val_score:.5f}{res}'
                    f' | Best iteration: {blu}{best_iter:>4}{res}')

                # permutation importance
                if permut:
                    perm = PermutationImportance(clf, scoring=None, n_iter=1, 
                                                random_state=42, cv=None, refit=False).fit(X_val, y_val)

                    perm_importance_df = pd.DataFrame({'importance': perm.feature_importances_}, 
                                                    index=X_val.columns).sort_index()

                    if perm_df_.shape[0] == 0:
                        perm_df_ = perm_importance_df.copy()
                    else:
                        perm_df_ += perm_importance_df

                # tree feature importance
                f_i = pd.DataFrame(sorted(zip(clf.feature_importances_, X.columns), 
                                                reverse=True, key=lambda x: x[1]), 
                                columns=['Value','Feature'])

                if feature_importances_.shape[0] == 0:
                    feature_importances_ = f_i.copy()
                else:

                    feature_importances_['Value'] += f_i['Value']

                # Boruta SHAP importance
                if boruta:
                    model = BoostBoruta(clf, importance_type='shap_importances', train_importance=False)
                    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric=bll_metric, verbose=0)

                    boruta_importance_df = pd.DataFrame({'importance': model.ranking_}, index=X_train.columns).sort_index()
                    if boruta_df_.shape[0] == 0:
                        boruta_df_ = boruta_importance_df.copy()
                    else:
                        boruta_df_ += boruta_importance_df

            fold_cv_score = metric(y_re, oof)
            print(f'{red} CV score: {res} {metric.__name__}: {red}{fold_cv_score:.5f}{res}')
            print(f'{"*" * 50}\n')
            cv_scores.append(fold_cv_score)


        print(f'{red} Avg score {CFG.n_feature_sel_folds}-fold: {res} {metric.__name__}: {red}{np.mean(cv_scores):.5f}{res}')
        print(f'{"*" * 50}\n')
        
        if permut:
            perm_df_ = perm_df_.sort_values('importance', ascending=False)
            
        if boruta:
            boruta_df_ = boruta_df_.sort_values('importance')
                                        
        feature_importances_ = feature_importances_.sort_values('Value', ascending=False)
        
        return perm_df_, feature_importances_, boruta_df_, np.mean(cv_scores)

    if CFG.feature_sel:
        perm_df_, feature_importances_, boruta_df_, cv_scores = lgbm_tuning(features, permut=True, boruta=True)

# Check features correlation

In [12]:
if CFG.feature_sel:
    col = 'DA'
    x = train_df[train_df[col] <= train_df[col].quantile(0.99)]
    cm = x[[c for c in train_df.columns if c not in ['Id', 'Class']]].corr()
    display(np.abs(cm[col]).sort_values(ascending=False)[1:])

# Analyze permutation feature importance

In [13]:
if CFG.feature_sel:
    perm_df_.to_csv('perm_df.csv')
    perm_df_
    perm_cols = set(perm_df_.index[-35:])
    display(perm_cols)

In [14]:
# x = pd.read_csv('perm_df.csv', index_col='Unnamed: 0')
# x['feature'] = x.index.copy()
# x = x.reset_index(drop=True)
# x['rank'] = x['importance'].rank()
# x = x[['feature', 'rank']]

# Analyze tree gain feature importance

In [15]:
if CFG.feature_sel:
    feature_importances_.to_csv('feature_importances.csv')
    feature_importances_
    fi_cols = set(feature_importances_['Feature'].values[-23:])
    display(fi_cols)

In [16]:
# y = pd.read_csv('feature_importances.csv')
# y['feature'] = y['Feature']
# y = y.reset_index(drop=True)
# y['rank'] = y['Value'].rank()
# y = y[['feature', 'rank']]

# Analyze BORUTA importance

In [17]:
if CFG.feature_sel:
    boruta_df_.to_csv('boruta_df_.csv')
    boruta_df_
    boruta_cols = set(boruta_df_.index[-35:])
    display(boruta_cols)

In [18]:
# z = pd.read_csv('boruta_df_.csv', index_col='Unnamed: 0')
# z['feature'] = z.index.copy()
# z = z.reset_index(drop=True)
# z['rank'] = z['importance'].rank(ascending=False)
# z = z[['feature', 'rank']]

# Sort all features according to their importance

In [19]:
# a = pd.concat([x, y, z])
# a = a[['feature', 'rank']]
# res = a.groupby('feature')['rank'].sum().sort_values(ascending=False)#.index.to_list()
# res

# LGBM Optuna optimization

In [20]:
def bll_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

X, y = train_df[features], train_df['Class'] 
best_iterations = list()
    
def optimize_model(params, how):
    bll_list = list()
    best_trial_iterations = list()
    
    for i in range(CFG.n_optimize_repeats):
        print(f'Repeat {blu}#{i+1}')

        # Make random under- or oversampling to balance classes
        if CFG.undersample:
            positive_count_train = train_df['Class'].value_counts()[1]
            sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train * class_imbalance, 
                                                            1: positive_count_train}, 
                                        random_state=15062023+i, 
                                        replacement=True)
        elif CFG.oversample:
            negative_count_train = train_df['Class'].value_counts()[0]
            sampler = RandomOverSampler(sampling_strategy={0: negative_count_train, 
                                                        1: negative_count_train // class_imbalance}, 
                                        random_state=2306020231)


        X_re, y_re = pd.concat([train_df[features], greeks.iloc[:,1:4]], axis=1), train_df['Class']
        
        if CFG.undersample:
            X_re, y_re = sampler.fit_resample(X_re, y_re)
        
        # Create Stratified Multilabel k-Fold scheme
        kf = MultilabelStratifiedKFold(n_splits=CFG.n_optimize_folds, shuffle=True, random_state=10062023+i)

        # Create an oof array for inner loop
        oof = np.zeros(X_re.shape[0])

        # Stratify based on Class and Alpha (3 types of conditions)
        for fold, (train_idx, val_idx) in enumerate(kf.split(X=X_re[features], y=X_re.iloc[:,-3:]), start=1): 
            X, y = X_re[features], y_re
            
            # Split the dataset according to the fold indexes.
            X_train = X.iloc[train_idx]
            X_val = X.iloc[val_idx]
            y_train = y.iloc[train_idx]
            y_val = y.iloc[val_idx]

            # oversample
            if CFG.oversample:
                X_train, y_train = sampler.fit_resample(X_train, y_train)

            if how == 'lgbm':
                model = lgb.LGBMClassifier(**params)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric=bll_metric, verbose=0)
                best_iter = model.best_iteration_
            elif how == 'xgboost':
                model = xgb.XGBClassifier(**params)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=0)
                best_iter = model.get_booster().best_iteration
            elif how == 'catboost':
                train_pool = Pool(X_train, y_train, cat_features=['EJ'])
                val_pool = Pool(X_val, y_val, cat_features=['EJ']) 
                model = cat.CatBoostClassifier(**params)
                model.fit(train_pool, eval_set=val_pool, verbose=0)
                best_iter = model.best_iteration_
            else:
                return 0

            val_preds = model.predict_proba(X_val)[:,1]
            oof[val_idx] = val_preds
        
        bll_list.append(balanced_log_loss(y_re, oof))
        best_trial_iterations.append(best_iter)
    
    best_iterations.append(int(np.mean(best_trial_iterations)))

    return np.mean(bll_list)

def objective(trial):
    params = {
        # Main parameters
#                     'device': 'gpu',
#                     'gpu_platform_id': 0,
#                     'gpu_device_id': 0,
        'objective': 'binary',
        'metric': 'none',
        'boosting_type': trial.suggest_categorical('boosting_type', ['goss']), # 'gbdt', 'dart']),   
        # Hyperparamters (in order of importance decreasing)
        'n_estimators': CFG.n_estimators, # trial.suggest_int('n_estimators', 500, 1500),  # max number of trees in model
        'early_stopping_round': CFG.early_stopping_rounds, 
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1,  alias: lambda_l1
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2, alias: lambda_l2
         # decrease to deal with overfit
        'max_depth': trial.suggest_int('max_depth', 5, 9),   # tree max depth 
         # decrease to deal with overfit
        'num_leaves': trial.suggest_int('num_leaves', 4, 128),  # Max number of leaves in one tree
                                                                # should be ~ 2**(max_depth-1)
        'subsample': None, # Randomly select part of data without 
                                  # resampling if subsample < 1.0
                                  # alias: bagging_fraction
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.7), # Randomly select a subset of features 
                                                                   # if colsample_bytree < 1.0
                                                                   # alias:feature_fraction
        # decrease to deal with overfit
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100), # Minimal number of data in one leaf
                                                                             # aliases: min_data_in_leaf, 
        # increase for accuracy, decrease to deal with overfit
        'max_bin': trial.suggest_int('max_bin', 32, 255), # Max number of bins that feature values will be bucketed in
        # increase to deal with overfit
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 7), # Perform bagging at every k iteration, alias: bagging_freq

#           'subsample_for_bin': 200000, # Number of data that sampled to construct feature discrete bins; setting this 
                                     # to larger value will give better training result but may increase train time 
#           'cat_smooth': trial.suggest_float('cat_smooth', 10.0, 100.0),  # this can reduce the effect of noises in 
                                                                       # categorical features, especially for 
                                                                       # categories with few data
        'verbose': -1
    }

    if not CFG.oversample and not CFG.undersample:
        params['is_unbalance'] = True
        params['class_weight'] = 'balanced'
    else:
        params['scale_pos_weight'] = class_imbalance
    
    if params['boosting_type'] != 'goss':
        params['subsample'] = trial.suggest_float('subsample', 0.3, 0.7)
    
    return optimize_model(params, how='lgbm')
            

if CFG.lgbm_optimize:
#     study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=100), direction="minimize")
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=CFG.n_trials * 2) 

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    df = study.trials_dataframe()
    df['n_estimators'] = best_iterations
    df = df.sort_values('value')
    df.to_csv(f'optuna_lgbm.csv')

    display(df.head(10))

[I 2023-06-29 13:14:06,334] A new study created in memory with name: no-name-0ee9eb82-519a-475e-8d2c-a48f7fe6d195


Repeat #1
Repeat #2


[I 2023-06-29 13:14:09,851] Trial 0 finished with value: 0.4071582514538665 and parameters: {'boosting_type': 'goss', 'learning_rate': 0.005048082013483599, 'reg_alpha': 2.818307318482172e-08, 'reg_lambda': 0.00020034812743883923, 'max_depth': 8, 'num_leaves': 91, 'colsample_bytree': 0.6576461630527592, 'min_child_samples': 51, 'max_bin': 252, 'subsample_freq': 4}. Best is trial 0 with value: 0.4071582514538665.


Repeat #1
Repeat #2


[I 2023-06-29 13:14:24,817] Trial 1 finished with value: 0.21017460085268774 and parameters: {'boosting_type': 'goss', 'learning_rate': 0.00394783907043978, 'reg_alpha': 0.00011752654746514142, 'reg_lambda': 2.2379888407845533e-05, 'max_depth': 6, 'num_leaves': 112, 'colsample_bytree': 0.5375863359991689, 'min_child_samples': 22, 'max_bin': 149, 'subsample_freq': 4}. Best is trial 1 with value: 0.21017460085268774.


...

[I 2023-06-29 14:18:20,980] Trial 998 finished with value: 0.19072650100618382 and parameters: {'boosting_type': 'goss', 'learning_rate': 0.04107925905241445, 'reg_alpha': 0.005156228268959003, 'reg_lambda': 0.0013040762474023804, 'max_depth': 6, 'num_leaves': 55, 'colsample_bytree': 0.5028987713235127, 'min_child_samples': 14, 'max_bin': 40, 'subsample_freq': 3}. Best is trial 137 with value: 0.17512873362467118.


Repeat #1
Repeat #2


[I 2023-06-29 14:18:25,994] Trial 999 finished with value: 0.1981879372849703 and parameters: {'boosting_type': 'goss', 'learning_rate': 0.016025808128231952, 'reg_alpha': 0.0037014689646844533, 'reg_lambda': 0.00047990875584815883, 'max_depth': 6, 'num_leaves': 54, 'colsample_bytree': 0.5184867228820171, 'min_child_samples': 8, 'max_bin': 191, 'subsample_freq': 3}. Best is trial 137 with value: 0.17512873362467118.


Number of finished trials: 1000
Best trial:
  Value: 0.17512873362467118
  Params: 
    boosting_type: goss
    learning_rate: 0.06287717371179619
    reg_alpha: 1.1716387435032611e-05
    reg_lambda: 2.0320606087186432e-08
    max_depth: 5
    num_leaves: 49
    colsample_bytree: 0.4901701605114531
    min_child_samples: 10
    max_bin: 41
    subsample_freq: 4


,number,value,datetime_start,datetime_complete,duration,params_boosting_type,params_colsample_bytree,params_learning_rate,params_max_bin,params_max_depth,params_min_child_samples,params_num_leaves,params_reg_alpha,params_reg_lambda,params_subsample_freq,state,n_estimators
137,137,0.175129,2023-06-29 13:19:50.160085,2023-06-29 13:19:52.208376,0 days 00:00:02.048291,goss,0.490170,0.062877,41,5,10,49,0.000012,2.032061e-08,4,COMPLETE,213
66,66,0.177956,2023-06-29 13:17:25.404702,2023-06-29 13:17:27.783037,0 days 00:00:02.378335,goss,0.585561,0.058126,40,6,10,56,0.000347,1.677651e-08,6,COMPLETE,292
101,101,0.178815,2023-06-29 13:18:33.552236,2023-06-29 13:18:35.218942,0 days 00:00:01.666706,goss,0.568894,0.070156,52,7,12,65,0.000014,9.223997e-08,6,COMPLETE,147
810,810,0.179267,2023-06-29 14:02:42.877305,2023-06-29 14:02:45.655523,0 days 00:00:02.778218,goss,0.626746,0.033533,40,5,11,56,0.000005,7.166722e-08,5,COMPLETE,392
116,116,0.179896,2023-06-29 13:19:02.935716,2023-06-29 13:19:04.785193,0 days 00:00:01.849477,goss,0.526095,0.058662,68,5,12,53,0.000032,7.565220e-08,5,COMPLETE,232
558,558,0.179965,2023-06-29 13:41:07.612988,2023-06-29 13:41:09.186893,0 days 00:00:01.573905,goss,0.498740,0.073318,54,5,9,59,0.000002,5.227462e-06,3,COMPLETE,163
324,324,0.180177,2023-06-29 13:27:55.749903,2023-06-29 13:27:57.497696,0 days 00:00:01.747793,goss,0.599607,0.062776,66,5,10,29,0.008491,2.248461e-06,3,COMPLETE,198
440,440,0.180546,2023-06-29 13:32:23.702855,2023-06-29 13:32:25.330168,0 days 00:00:01.627313,goss,0.483841,0.036756,49,5,8,28,0.000035,1.546537e-08,3,COMPLETE,321
882,882,0.181092,2023-06-29 14:08:32.818637,2023-06-29 14:08:35.182129,0 days 00:00:02.363492,goss,0.383318,0.041468,54,8,12,56,0.000077,2.581422e-08,4,COMPLETE,300
877,877,0.181142,2023-06-29 14:08:12.192405,2023-06-29 14:08:14.340726,0 days 00:00:02.148321,goss,0.476130,0.041866,54,5,11,55,0.000008,6.822503e-08,4,COMPLETE,244


# Load LGBM parameters

In [21]:
import glob

if CFG.kaggle:
    param_list = glob.glob("/kaggle/input/icr-optuna-no-da/optuna_lgbm.csv")
else:
    param_list = glob.glob("optuna_lgbm.csv")

models = list()
best_lgbm_params = list()

lgbm_params = pd.DataFrame()

for f in param_list:
    tmp = pd.read_csv(f, index_col='Unnamed: 0')
    if lgbm_params.shape[0] == 0:
        lgbm_params = tmp
    else:
        lgbm_params = pd.concat([lgbm_params, tmp])
        
lgbm_params = lgbm_params.sort_values('value').head(CFG.n_stacking_models_lgbm)
param_cols = [c for c in lgbm_params.columns if c.startswith('params_')]
lgbm_params = lgbm_params[param_cols]

for idx, row in lgbm_params.iterrows():
    row_dict = {k[7:]: v for k, v in row.items()}
    row_dict['objective'] = 'binary'
    row_dict['metric'] = 'none'
#     row_dict['subsample_for_bin'] = 300000
    row_dict['force_col_wise'] = False
    row_dict['early_stopping_round'] = CFG.early_stopping_rounds
    row_dict['verbose'] = -1
    row_dict['max_bin'] = 255
    
    row_dict['n_estimators'] = CFG.n_estimators# int(row_dict['n_estimators'])
    row_dict['num_leaves'] = int(row_dict['num_leaves'])
    row_dict['max_depth'] = int(row_dict['max_depth'])
    row_dict['min_child_samples'] = int(row_dict['min_child_samples'])
    row_dict['subsample_freq'] = int(row_dict['subsample_freq'])
    row_dict['learning_rate'] = float(row_dict['learning_rate'])
    
    if not CFG.oversample and not CFG.undersample:
        row_dict['is_unbalance'] = True
        row_dict['class_weight'] = 'balanced'
    else:
        row_dict['scale_pos_weight'] = class_imbalance
    
    if row_dict['boosting_type'] == 'goss':
        row_dict['subsample'] = None
        
    best_lgbm_params.append(row_dict)

if CFG.test:
    best_lgbm_params = [{
            'boosting_type': 'goss',
            'n_estimators': 50000,
            'early_stopping_round': 300,
            'max_depth': 8,
            'learning_rate': 0.06733232950390658,
            'subsample': 0.6970532011679706,
            'colsample_bytree': 0.6055755840633003,
            'is_unbalance': True, 
            'class_weight': 'balanced',
            'metric':'none',
            'verbose': -1,
            'random_state': 42,
        }
    ]

                         

# XGBoost Optuna optimization

In [22]:
X, y = train_df[features], train_df['Class']

def objective(trial):
    bll_list = list()

    params = {
        "n_estimators": CFG.n_estimators, # trial.suggest_int('n_estimators', 100, 1000, step=100),
        "early_stopping_rounds": CFG.early_stopping_rounds,
        "verbosity": 0,
        "random_state": 14062023,
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree"]),# "dart", "gblinear"]), 
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
    }

    # if not CFG.oversample and not CFG.undersample:
    params["scale_pos_weight"] = class_imbalance
    
    if params["booster"] in ["gbtree", "dart"]:
        params["learning_rate"] = trial.suggest_float("learning_rate", 1e-4, 0.1, log=True) # alias eta
        # maximum depth of the tree, signifies complexity of the tree.
        params["max_depth"] = trial.suggest_int("max_depth", 3, 10)
        # minimum child weight, larger the term more conservative the tree.
        params["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        # defines how selective algorithm is.
        params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if params["booster"] == "dart":
        params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        params["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        params["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    return optimize_model(params, how='xgboost')

if CFG.xgb_optimize:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=CFG.n_trials * 2)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    df = study.trials_dataframe().sort_values('value')
    df.to_csv(f'optuna_xgb.csv')

    display(df.head(10))

# Load XGBoost parameters

In [23]:
import glob

if CFG.kaggle:
    param_list = glob.glob("/kaggle/input/icr-optuna-no-da/optuna_xgb.csv")
else:
    param_list = glob.glob("optuna_xgb.csv")

models = list()
best_xgb_params = list()

xgb_params = pd.DataFrame()

for f in param_list:
    tmp = pd.read_csv(f, index_col='Unnamed: 0')
    if xgb_params.shape[0] == 0:
        xgb_params = tmp
    else:
        xgb_params = pd.concat([xgb_params, tmp])
        
xgb_params = xgb_params.sort_values('value').head(CFG.n_stacking_models_xgb)
param_cols = [c for c in xgb_params.columns if c.startswith('params_')]
xgb_params = xgb_params[param_cols]

for idx, row in xgb_params.iterrows():
    row_dict = {k[7:]: v for k, v in row.items()}
    row_dict['n_estimators'] = CFG.n_estimators
    row_dict['early_stopping_rounds'] = CFG.early_stopping_rounds
    row_dict['random_state'] = 14062023
    row_dict['verbosity'] = 0
    row_dict['objective'] = "binary:logistic"
    row_dict['eval_metric'] = "logloss"
    row_dict['tree_method'] = "exact"
    row_dict['booster'] = "gbtree"

    # if not CFG.oversample and not CFG.undersample:
    row_dict['scale_pos_weight'] = class_imbalance

    if row_dict["booster"] in ["gbtree", "dart"]:
        row_dict["max_depth"] = int(row_dict["max_depth"])
        row_dict["min_child_weight"] = int(row_dict["min_child_weight"])
    else:
        row_dict["learning_rate"] = None
        row_dict["max_depth"] = None
        row_dict["min_child_weight"] = None
        row_dict["gamma"] = None
        row_dict["grow_policy"] = None     

    if row_dict["booster"] != "dart":
        row_dict["sample_type"] = None
        row_dict["normalize_type"] = None
        row_dict["rate_drop"] = None
        row_dict["skip_drop"] = None

    best_xgb_params.append(row_dict)

if CFG.test:
    best_xgb_params = [{
            'n_estimators': CFG.n_estimators,
            'early_stopping_rounds': CFG.early_stopping_rounds,
            'objective': "binary:logistic",
            'scale_pos_weight': class_imbalance, 
            'verbosity': 0,
            'random_state': 19062023,
        }
    ]

# CatBoost Optuna optimization

In [24]:
X, y = train_df[features], train_df['Class']

def objective(trial):
    
    bll_list = list()

    # Parameters
    params = {
        'task_type': 'CPU', # GPU
        'eval_metric': 'Logloss',
        'loss_function': 'Logloss', 
        'random_seed': 19062023,
        'od_type': 'Iter', # Type of overfitting detector - stop after k iteraions
        'iterations' : CFG.n_estimators, # trial.suggest_int('iterations', 300, 1200),        
        'od_wait': CFG.early_stopping_rounds, # Overfitting detector - stop training after k iterations without metric improvement
        # 'metric_period': 100, # Show metric each k iterations
        # Hyperparamters (in order of importance decreasing)
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS', 'No']),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 3e-1), 
        'l2_leaf_reg': trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
        'depth' : trial.suggest_int('depth', 4, 10),  # Max tree depth                                          
         # increase to deal with overfit
        'random_strength': trial.suggest_float('random_strength', 0, 100), # The amount of randomness to use
                                                                           # for scoring splits when the tree structure
                                                                           # is selected. Helps to avoid overfitting
                                                                           # CPU only
        # per_float_feature_quantization='0:border_count=1024'
        'border_count': 254, # trial.suggest_categorical('border_count', [128, 254]), # The number of splits for numerical features
                                                                                      # bigger is better but slowly
                                                                                      # alias: max_bin
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100), # Minimal number of data in one leaf
                                                                           # aliases: min_child_samples, 

    }

    if not CFG.oversample and not CFG.undersample:
        params['scale_pos_weight'] = class_imbalance
    else:
        params['auto_class_weights'] = 'Balanced'
        
    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 100) # Assigns random 
                                                                                           # weights to objects
                                                                                           # works only with 
                                                                                           # Bayesian bootstrap
    if params["bootstrap_type"] in ["Poisson", "Bernoulli", "MVS"]:
        params["subsample"] = trial.suggest_float("subsample", 0.3, 1) # Percentage of objects to use 
                                                                        # at each split

    if params['task_type'] == 'CPU' and params['bootstrap_type'] != 'Bayesian':
        params["colsample_bylevel"] = trial.suggest_float("colsample_bylevel", 0.3, 1)  # Percentage of features to use 
                                                                                        # at each split;
                                                                                        # with Bayesian bootstrap and Lossguide grop policy
                                                                                        # leads to error (CatBoost bug)
    else:
        params["colsample_bylevel"] = None                                                     

    if params['grow_policy'] == 'Lossguide': 
        params['max_leaves'] = trial.suggest_int('max_leaves', 4, 128) # Max number of leaves in one tree 
                                                                       # decrease to deal with the overfit

    if params['grow_policy'] == 'SymmetricTree': 
        params['boosting_type'] = trial.suggest_categorical('boosting_type', ['Ordered', 'Plain'])
    else:
        params['boosting_type'] = 'Plain'
    
    return optimize_model(params, how='catboost')

if CFG.cb_optimize:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=CFG.n_trials)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    df = study.trials_dataframe().sort_values('value')
    df.to_csv(f'optuna_catboost.csv')

    display(df.head(10))

# Load CatBoost parameters

In [25]:
import glob

if CFG.kaggle:
    param_list = glob.glob("/kaggle/input/icr-optuna-no-da/optuna_catboost.csv")
else:
    param_list = glob.glob("optuna_catboost.csv")

models = list()
best_cb_params = list()

cb_params = pd.DataFrame()

for f in param_list:
    tmp = pd.read_csv(f, index_col='Unnamed: 0')
    if cb_params.shape[0] == 0:
        cb_params = tmp
    else:
        cb_params = pd.concat([cb_params, tmp])
        
cb_params = cb_params.sort_values('value').head(CFG.n_stacking_models_cb)
param_cols = [c for c in cb_params.columns if c.startswith('params_')]
cb_params = cb_params[param_cols]


for idx, row in cb_params.iterrows():
    row_dict = {k[7:]: v for k, v in row.items()}
    row_dict['task_type'] = 'CPU'
    row_dict['eval_metric'] = 'Logloss'
    row_dict['loss_function'] = 'Logloss'
    row_dict['random_seed'] = 13062023
    row_dict['verbose'] = 0
    row_dict['od_type'] = 'Iter'
    row_dict['iterations'] = CFG.n_estimators * 4
    row_dict['od_wait'] = CFG.early_stopping_rounds
    row_dict['border_count'] = 254
    
    if not CFG.oversample and not CFG.undersample:
        row_dict['scale_pos_weight'] = class_imbalance
    else:
        row_dict['auto_class_weights'] = 'Balanced'
        
    if row_dict["task_type"] != "GPU":
        row_dict['colsample_bylevel'] = None
    
    if row_dict["bootstrap_type"] != "Bayesian":
        row_dict['bagging_temperature'] = None
        
    if row_dict["bootstrap_type"] not in ["Poisson", "Bernoulli", "MVS"]:
        row_dict['subsample'] = None
    
    if row_dict['grow_policy'] == 'Lossguide':
        row_dict['max_leaves'] = int(row_dict['max_leaves'])
    else:
        row_dict['max_leaves'] = None
    
    if row_dict['grow_policy'] != 'SymmetricTree':
        row_dict['boosting_type'] = 'Plain'
    
    best_cb_params.append(row_dict)

if CFG.test:
    best_cb_params = [{
            'iterations': CFG.n_estimators,
            'od_type': 'Iter',
            'od_wait': CFG.early_stopping_rounds,
            'eval_metric': "Logloss",
            'loss_function': "Logloss",
            'auto_class_weights': 'Balanced', 
            'verbose': 0,
            'random_seed': 19062023,
        }
    ]

# Train models

In [34]:
def bll_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

def pp_prob(p):
    c0 = p[:,0].sum()
    c1 = p[:,1:].sum()
    new_p = p * np.array([[1/(c0 if i==0 else c1) for i in range(p.shape[1])]])
    new_p = new_p / np.sum(new_p,axis=1,keepdims=1)
    return np.sum(new_p[:,1:],1,keepdims=False)

def model_train(how, best_params):

    oof_level2 = np.zeros([train_df['Class'].shape[0], len(best_params) + 1])
    oof_level2[:, len(best_params)] = train_df['Class']
    oof_level2_test = np.zeros([test_df.shape[0], len(best_params)])
    
    for i, params in tqdm(enumerate(best_params), total=len(best_params)):
        
        if how == 'tabpfn':
            X, y, test = train_df_tabpfn[num_cols], train_df['Class'], test_df_tabpfn[num_cols]
        else:
            X, y, test = train_df[features], train_df['Class'], test_df[features]
    
        kf = MultilabelStratifiedKFold(n_splits=CFG.n_stacking_folds, shuffle=True, random_state=80620231+i)

        print(f"Training with {blu}{len(features)}{res} features")

        for fold, (fit_idx, val_idx) in enumerate(kf.split(X=X, y=greeks.iloc[:,1:4]), start=1):
            # Split the dataset according to the fold indexes.
            X_train = X.iloc[fit_idx]
            X_val = X.iloc[val_idx]
            y_train = y.iloc[fit_idx]
            y_val = y.iloc[val_idx]

            # Make random under- or oversampling to balance classes
            if CFG.undersample or CFG.oversample:
                if CFG.undersample:
                    positive_count_train = y_train.value_counts()[1]
                    sampler = RandomUnderSampler(sampling_strategy={0: positive_count_train * class_imbalance, 
                                                                    1: positive_count_train}, 
                                                random_state=15062023+i, 
                                                replacement=True)
                elif CFG.oversample:
                    negative_count_train = y_train.value_counts()[0]
                    sampler = RandomOverSampler(sampling_strategy={0: negative_count_train, 
                                                                   1: negative_count_train // class_imbalance}, 
                                                random_state=2306020231+i)

                X_train, y_train = sampler.fit_resample(X_train, y_train)
            
            if how == 'lgbm':
                model = lgb.LGBMClassifier(**params)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric=bll_metric, verbose=0)
                best_iter = model.best_iteration_
            elif how == 'xgboost':
                model = xgb.XGBClassifier(**params)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=0)
                best_iter = model.get_booster().best_iteration
            elif how == 'catboost':
                train_pool = Pool(X_train, y_train, cat_features=['EJ'])
                val_pool = Pool(X_val, y_val, cat_features=['EJ'])   
                model = cat.CatBoostClassifier(**params)
                model.fit(train_pool, eval_set=val_pool, verbose=0)
                best_iter = model.best_iteration_
            elif how == 'tabpfn':
                model = TabPFNClassifier(N_ensemble_configurations=64, device='cuda:0')
                model.fit(X_train, y_train, overwrite_warning=True)
                best_iter = 0
            else:
                return None, None
                
            if how == 'tabpfn':
                val_preds = pp_prob(model.predict_proba(X_val))
                oof_level2_test[:, i] += pp_prob(model.predict_proba(test))
            else:
                val_preds = model.predict_proba(X_val)[:,1]
                oof_level2_test[:, i] += model.predict_proba(test)[:,1]
            
            oof_level2[val_idx, i] = val_preds

            val_score = balanced_log_loss(y_val, val_preds)
            
            print(f'Fold: {blu}{fold:>3}{res}| bll_metric: {blu}{val_score:.5f}{res}'
                    f' | Best iteration: {blu}{best_iter:>4}{res}') 
        
    return oof_level2, oof_level2_test / CFG.n_stacking_folds

oof_train_list = list()
oof_test_list = list()

if CFG.lgbm_train:
    oof_level2_lgbm, oof_level2_test_lgbm = model_train('lgbm', best_lgbm_params)
    oof_train_list.append(oof_level2_lgbm[:,:-1])
    oof_test_list.append(oof_level2_test_lgbm)
    y = oof_level2_lgbm[:,-1]

if CFG.xgb_train:
    oof_level2_xgb, oof_level2_test_xgb = model_train('xgboost', best_xgb_params)
    oof_train_list.append(oof_level2_xgb[:,:-1])
    oof_test_list.append(oof_level2_test_xgb)
    y = oof_level2_xgb[:,-1]

if CFG.cb_train:
    oof_level2_cb, oof_level2_test_cb = model_train('catboost', best_cb_params)
    oof_train_list.append(oof_level2_cb[:,:-1])
    oof_test_list.append(oof_level2_test_cb)
    y = oof_level2_cb[:,-1]

if CFG.tabpfn_train:
    oof_level2_tabpfn, oof_level2_test_tabpfn = model_train('tabpfn', [i for i in range(CFG.n_stacking_models_tabpfn)])
    oof_train_list.append(oof_level2_tabpfn[:,:-1])
    oof_test_list.append(oof_level2_test_tabpfn)
    y = oof_level2_tabpfn[:,-1]

  0%|          | 0/40 [00:00<?, ?it/s]

Training with 38 features
Fold:   1| bll_metric: 0.16397 | Best iteration:  181
Fold:   2| bll_metric: 0.24840 | Best iteration:   90
Fold:   3| bll_metric: 0.07244 | Best iteration:  496
Fold:   4| bll_metric: 0.25533 | Best iteration:  151
Fold:   5| bll_metric: 0.32206 | Best iteration:   86
Training with 38 features
Fold:   1| bll_metric: 0.28925 | Best iteration:   82
Fold:   2| bll_metric: 0.22941 | Best iteration:  199
Fold:   3| bll_metric: 0.18174 | Best iteration:  160
Fold:   4| bll_metric: 0.08665 | Best iteration:  588
Fold:   5| bll_metric: 0.16644 | Best iteration:  164
Training with 38 features
Fold:   1| bll_metric: 0.13258 | Best iteration:  106
Fold:   2| bll_metric: 0.22311 | Best iteration:  136
Fold:   3| bll_metric: 0.14743 | Best iteration:  311
Fold:   4| bll_metric: 0.23914 | Best iteration:  102
Fold:   5| bll_metric: 0.11555 | Best iteration:  183
Training with 38 features
Fold:   1| bll_metric: 0.14749 | Best iteration:  338
Fold:   2| bll_metric: 0.14627 |

# Blending and Stacking with Logistic Regression

In [35]:
from sklearn.linear_model import LogisticRegression

oof_level2 = np.concatenate(oof_train_list, axis=1)
oof_level2_test = np.concatenate(oof_test_list, axis=1)

X = oof_level2

# mean bll
print(balanced_log_loss(y, np.mean(X, axis=1)))

lr = LogisticRegression(class_weight='balanced')
lr.fit(X, y)

pred = lr.predict_proba(X)[:,1]

# lr bll
print(balanced_log_loss(y, pred))

weights = lr.coef_[0]

0.17796230631747223
0.1391273380256587


In [28]:
# LGBM full dataset
# 0.13372873188516746
# 0.09363332625161021

# LGBM full dataset with adjusted scaling
# 0.13467176782918622
# 0.09374656290880284

# full dataset with class balance 3
# 0.18130449603081458
# 0.13249114243596113

# undersample with class balance 3
# 0.11174849200860162
# 0.09973545010826133

# undersample with class balance 3 and different undersamping for every model (validatidation was not undersampled)
# 0.19671222032350008 / CV ~ 0.21
# 0.12812153512778934

# oversample with class balance 3
# 0.17393849284307183
# 0.11238039356223456

# oversample with class balance 3 and different undersamping for every model (validatidation was not undersampled)
# 0.17459339388826078
# 0.11296531409464912


In [29]:
# LGBM + CatBoost + XGBoost + TabPFN
# 0.17472710249039772
# 0.09683636947851168

# LGBM oversampled 10 folds
# 0.1899300127485599
# 0.1321074079639491

# LGBM + CatBoost + XGBoost + TabPFN + different k-fold for every model
# 0.17616782811158316
# 0.09637187593530008

# LGBM + CatBoost + XGBoost + TabPFN + different k-fold for every model + 20 folds
# 0.16113327662498975
# 0.08182251667662065

# LGBM 20 folds
# 0.13372873188516746
# 0.09363332625161021

# Which objects are the most erroneus?

In [30]:
preds = np.mean(X, axis=1)
errors = pd.Series(np.abs(y - preds))
errors = errors.sort_values(ascending=False) 
errors[errors >= errors.quantile(0.99)].index.to_list()


[292, 102, 509, 313, 380, 337, 367]

# Find the best class threshold

In [31]:
def pp_prob3(_oof, _p, num=1.5):
    # increase (num > 1) or decrease (num < 1) binary prediction value
    oof = num * _oof / ((num - 1) * _oof + 1)
    p = num * _p / ((num - 1) * _p + 1)
    return oof, p

def inflate_preds(_y, _oof, _p):
    # find the best num multiplier for binary prediction
    best_score = np.inf
    best_num = None
    best_oof = None
    best_p = None
    
    candidates = np.linspace(0.05,5,100)
    for num in candidates:
        curr_oof, curr_p = pp_prob3(_oof, _p, num)
        curr_score = balanced_log_loss(_y, curr_oof)
        if curr_score < best_score:
            best_num = num
            best_score = curr_score
            best_p = curr_p
            best_oof = curr_oof
    print('best num:', round(best_num, 2), '/ best score:', best_score)
    return best_oof, best_p

# Predict test

In [32]:
def predict(X):
    y = np.zeros(len(X))
    for i in range(oof_level2_test.shape[1]):
        # y += weights[i] * oof_level2_test[:,i]
        y += oof_level2_test[:,i]
    # return y / sum(weights)
    return y / oof_level2_test.shape[1]

predictions = predict(test_df[features])

if CFG.adjust_class_threshold:
    _, predictions = inflate_preds(y, np.mean(X, axis=1), predictions)

test_df['class_1'] = predictions
test_df['class_0'] = 1 - predictions

sample_submission[['class_0', 'class_1']] = test_df[['class_0', 'class_1']]
sample_submission.to_csv(r"submission.csv", index=False)
sample_submission

,Id,class_0,class_1
0,00eed32682bb,0.7518,0.2482
1,010ebe33f668,0.7518,0.2482
2,02fa521e1838,0.7518,0.2482
3,040e15f562a2,0.7518,0.2482
4,046e85c7cc7f,0.7518,0.2482
